In [1]:
import sys
import math
import json
import pandas as pd
import cobra
import cplex
import optlang
from configparser import ConfigParser
config = ConfigParser()
config.read("config.cfg")
paths = config.get("script", "syspaths").split(";")
for path in paths:
    sys.path.append(path)
import cobrakbase
from IPython.core.display import HTML
from modelseedpy import MSMedia, MSPackageManager, MSExpression

cobrakbase 0.2.8


In [2]:
model= cobra.io.read_sbml_model("iML1515.xml")
model.solver = 'optlang-cplex'
media_dict = {
    "glc__D":22.5,
    "nh4":100,
    "k":100,
    "pydxn":0.00002,
    'so4':100,
    'pi':100,
    'mn2': 100,
    'fe2': 100,
    'na1': 100,
    'mg2': 100,
    'o2': 20,
    'h2o': 100,
    'h': 100,
    'fe3': 100.0,
    'zn2': 100.0,
    'ca2': 100.0,
    'ni2': 100.0,
    'cu2': 100.0,
    'sel': 100.0,
    'cobalt2': 100.0,
    'mobd': 100.0,
    'cl': 100.0,
    'cbl1':100,
    'tungs': 100.0,
    'slnt': 100.0
}
media = MSMedia.from_dict(media_dict)
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
#Plug in growth rate and minimize nutrient uptake
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound = 0.3768
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound = 0.3768
glucose_uptake_objective = model.problem.Objective(
    1 * model.reactions.EX_glc__D_e.flux_expression,
    direction='max')
model.objective = glucose_uptake_objective
solution = model.optimize()
#Constraining media to limit glucose to hit measured growth rate
media_dict["glc__D"] = -1*solution.fluxes["EX_glc__D_e"]
print(media_dict["glc__D"])
media = MSMedia.from_dict(media_dict)
pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
#Confirming that we now get the correct growth rate
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.lower_bound = 0
model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.upper_bound = 100
biomass_objective = model.problem.Objective(
    1 * model.reactions.BIOMASS_Ec_iML1515_WT_75p37M.flux_expression,
    direction='max')
model.objective = biomass_objective
solution = cobra.flux_analysis.pfba(model)
print(solution.fluxes["BIOMASS_Ec_iML1515_WT_75p37M"])

4.447533484208366
0.3768000000000051


In [3]:
#Add the gene expression profile to the project notebook folder (Fatime)
expression_data = MSExpression.from_gene_feature_file("differentialexpression_gene.tsv",None,False)
#Covert differential gene expression profile into differential reaction expression profile (Chris)
reaction_expression = expression_data.build_reaction_expression(model,0)
#Need to load the metabolite differential profile data in terms of iML1515 IDs
metabolite_data = MSExpression.from_gene_feature_file("Metabolomics.tsv",None,False)
#Translate data into goals
goalflux = {}
reactions = ["4HTHRK","THRS"]
for rxn in reactions:
    if solution.fluxes[rxn] == 0:
        goalflux[rxn] = 1
    else:
        goalflux[rxn] = 1.5*solution.fluxes[rxn]
pkgmgr.getpkg("BilevelPkg").build_package(None,7)
pkgmgr.getpkg("ChangeOptPkg").build_package({
    "4HTHRK":{"flux":goalflux["4HTHRK"]*1.2,"direction":"high","objcoef":1},
    "THRS":{"flux":goalflux["THRS"]*1.2,"direction":"high","objcoef":1}
})
with open("multiomics.lp", 'w') as out:
    out.write(str(model.solver))
    out.close()
model.optimize()

binary_variable_count: 7
4HTHRK
FOUND!
THRS
FOUND!


,fluxes,reduced_costs
CYTDK2,0.000000,NaN
XPPT,0.000000,NaN
HXPRT,0.000000,NaN
NDPK5,-0.009347,NaN
SHK3Dr,0.140141,NaN
...,...,...
MPTS,0.000005,NaN
MOCOS,0.000005,NaN
BMOGDS2,0.000000,NaN
FESD2s,0.000000,NaN
